In [32]:
# Initial imports
import numpy as np
import datetime as dt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import hvplot.pandas
import os
from pathlib import Path
import lstm_functions

# 1. Data Preprocessing

In [3]:
#Read in the clean data
clean_df = pd.read_csv("clean_data\SPY_TR_VIX_data.csv", parse_dates=True, index_col='date')
clean_df

,Unnamed: 0,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap
date,,,,,,,,
2015-12-01,0,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410
2015-12-02,1,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055
2015-12-03,2,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936
2015-12-04,3,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631
2015-12-07,4,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128
...,...,...,...,...,...,...,...,...
2022-03-24,1589,445.82,450.5000,444.7600,450.48,61411569,526865,447.609014
2022-03-25,1590,451.09,452.9800,448.4300,452.69,77199328,661033,451.047221
2022-03-28,1591,452.04,455.9100,450.0600,455.91,68356914,620868,453.294974


In [4]:
clean_df['SPY_close'].hvplot()

:Curve   [date]   (SPY_close)

In [25]:
#Fetch cleaned data with features
merged_df = pd.read_csv('clean_data/basic_spy_vix.csv', parse_dates=True)
merged_df = merged_df.drop(columns=['Unnamed: 0', 'weekly_range'])
merged_df

,pct_change,day_range,pct_changeVIX,mean_volume,VIX_close,tr_weekly,tr_5,tr_project,day_range_VIX
0,-0.004446,2.5295,0.000908,272.286495,15.84,18.757144,0.00000,0.000000,1.60
1,0.002470,2.5090,-0.000436,266.573402,17.60,18.757144,3.31948,3.094519,1.81
2,-0.004219,4.5000,0.007565,277.036449,19.61,18.757144,3.66548,3.110070,4.41
3,0.002044,2.2900,0.000438,286.743585,19.34,18.757144,3.24370,3.091081,1.59
4,-0.007375,4.3500,0.014898,315.219794,24.39,18.757144,3.23570,3.090717,4.39
...,...,...,...,...,...,...,...,...,...
1050,-0.009454,8.7600,0.001758,106.291299,31.77,18.757144,10.04800,3.360554,3.12
1051,0.015513,9.8400,-0.007864,113.201143,29.83,18.757144,9.59800,3.344808,4.26
1052,0.013048,10.8800,-0.005466,102.762980,26.67,18.757144,9.12200,3.327879,3.51
1053,0.016993,7.8800,-0.001937,127.748564,25.67,18.757144,9.32000,3.334956,2.22


In [24]:
#Fetch all the indicator data
indicators_df = pd.read_csv('clean_data/indicators.csv', parse_dates=True)
indicators_df= indicators_df.drop(columns='Unnamed: 0')
indicators_df

,SPY_open,SPY_high,SPY_low,SPY_close,ATR,VBM,MI,KAMA,BBANDS Upper,Moving Average,BBands Lower,APZ Upper,APZ Lower,KC Upper,KC Lower,DYMI,weekly_range
0,209.20,209.7295,207.20,208.27,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,215.063501,201.382255,0.000000,0.000000,0.000000,0.0000
1,206.48,208.2890,205.78,206.99,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,214.468463,200.998387,0.000000,0.000000,0.000000,0.0000
2,206.20,208.6800,204.18,205.33,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,213.602554,200.045668,0.000000,0.000000,0.000000,0.0000
3,205.44,207.4300,205.14,205.86,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,213.154767,199.701220,0.000000,0.000000,0.000000,0.0000
4,203.38,204.1400,201.51,201.88,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,211.579142,198.274402,0.000000,0.000000,0.000000,8.2195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,420.98,424.5500,415.79,417.00,10.399414,-47.075725,23.258761,433.272462,448.902424,430.9365,412.970576,440.366720,402.385804,450.071391,410.838671,41.516734,0.0000
1051,419.66,426.8400,418.42,426.17,10.251557,-47.941947,23.453097,433.197594,447.959303,430.2940,412.628697,440.321976,402.514054,449.639293,410.454573,41.338068,0.0000
1052,429.94,435.6800,424.80,435.55,9.734414,-45.627367,23.702630,433.215559,446.018008,429.7665,413.514992,441.975807,404.263649,450.391395,410.750675,49.047465,0.0000
1053,433.70,441.0700,433.19,441.07,9.584414,-43.459943,23.985301,433.316946,444.689744,429.4900,414.290256,444.377769,406.795929,451.505296,411.636576,77.052721,0.0000


In [26]:
#Concatenate the two data frames to get all features and target column
combined_df = pd.concat([merged_df, indicators_df], axis='columns', join='inner')
combined_df

,pct_change,day_range,pct_changeVIX,mean_volume,VIX_close,tr_weekly,tr_5,tr_project,day_range_VIX,SPY_open,...,KAMA,BBANDS Upper,Moving Average,BBands Lower,APZ Upper,APZ Lower,KC Upper,KC Lower,DYMI,weekly_range
0,-0.004446,2.5295,0.000908,272.286495,15.84,18.757144,0.00000,0.000000,1.60,209.20,...,0.000000,0.000000,0.0000,0.000000,215.063501,201.382255,0.000000,0.000000,0.000000,0.0000
1,0.002470,2.5090,-0.000436,266.573402,17.60,18.757144,3.31948,3.094519,1.81,206.48,...,0.000000,0.000000,0.0000,0.000000,214.468463,200.998387,0.000000,0.000000,0.000000,0.0000
2,-0.004219,4.5000,0.007565,277.036449,19.61,18.757144,3.66548,3.110070,4.41,206.20,...,0.000000,0.000000,0.0000,0.000000,213.602554,200.045668,0.000000,0.000000,0.000000,0.0000
3,0.002044,2.2900,0.000438,286.743585,19.34,18.757144,3.24370,3.091081,1.59,205.44,...,0.000000,0.000000,0.0000,0.000000,213.154767,199.701220,0.000000,0.000000,0.000000,0.0000
4,-0.007375,4.3500,0.014898,315.219794,24.39,18.757144,3.23570,3.090717,4.39,203.38,...,0.000000,0.000000,0.0000,0.000000,211.579142,198.274402,0.000000,0.000000,0.000000,8.2195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,-0.009454,8.7600,0.001758,106.291299,31.77,18.757144,10.04800,3.360554,3.12,420.98,...,433.272462,448.902424,430.9365,412.970576,440.366720,402.385804,450.071391,410.838671,41.516734,0.0000
1051,0.015513,9.8400,-0.007864,113.201143,29.83,18.757144,9.59800,3.344808,4.26,419.66,...,433.197594,447.959303,430.2940,412.628697,440.321976,402.514054,449.639293,410.454573,41.338068,0.0000
1052,0.013048,10.8800,-0.005466,102.762980,26.67,18.757144,9.12200,3.327879,3.51,429.94,...,433.215559,446.018008,429.7665,413.514992,441.975807,404.263649,450.391395,410.750675,49.047465,0.0000
1053,0.016993,7.8800,-0.001937,127.748564,25.67,18.757144,9.32000,3.334956,2.22,433.70,...,433.316946,444.689744,429.4900,414.290256,444.377769,406.795929,451.505296,411.636576,77.052721,0.0000


In [76]:
#Save combined df with indicators and other features into one csv file
combined_df.to_csv('clean_data/combined.csv')

In [80]:
#Use the combined df with dates as index
combined_df_2 =pd.read_csv('clean_data/combined_dates.csv', index_col='date')
combined_df_2.drop(columns='Unnamed: 0', inplace=True)
combined_df_2

,pct_change,day_range,pct_changeVIX,mean_volume,VIX_close,tr_weekly,tr_5,tr_project,day_range_VIX,SPY_open,...,KC Lower,DYMI,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,weekly_range
date,,,,,,,,,,,,,,,,,,,,,
2015-12-07,-0.004446,2.5295,0.000908,272.286495,15.84,18.757144,0.00000,0.000000,1.60,209.20,...,0.000000,0.000000,102027111,374705,208.276128,2.5295,15.65,17.18,15.58,0.0000
2015-12-08,0.002470,2.5090,-0.000436,266.573402,17.60,18.757144,3.31948,3.094519,1.81,206.48,...,0.000000,0.000000,103372367,387782,206.966276,2.5090,17.69,18.33,16.52,0.0000
2015-12-09,-0.004219,4.5000,0.007565,277.036449,19.61,18.757144,3.66548,3.110070,4.41,206.20,...,0.000000,0.000000,162401537,586210,206.034646,4.5000,18.05,20.13,15.72,0.0000
2015-12-10,0.002044,2.2900,0.000438,286.743585,19.34,18.757144,3.24370,3.091081,1.59,205.44,...,0.000000,0.000000,116128858,404992,206.102975,2.2900,19.25,19.72,18.13,0.0000
2015-12-11,-0.007375,4.3500,0.014898,315.219794,24.39,18.757144,3.23570,3.090717,4.39,203.38,...,0.000000,0.000000,211173305,669924,203.150102,4.3500,21.36,25.27,20.88,8.2195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-11,-0.018986,9.2400,0.000747,112.222065,30.75,18.757144,10.88436,3.389176,2.20,428.20,...,412.506991,44.616020,90803923,809145,424.040193,9.2400,30.43,31.04,28.84,17.1818
2022-03-14,-0.009454,8.7600,0.001758,106.291299,31.77,18.757144,10.04800,3.360554,3.12,420.98,...,410.838671,41.516734,91251505,858504,419.220077,8.7600,31.03,33.18,30.06,0.0000
2022-03-15,0.015513,9.8400,-0.007864,113.201143,29.83,18.757144,9.59800,3.344808,4.26,419.66,...,410.454573,41.338068,104219651,920659,422.752423,9.8400,33.13,33.83,29.57,0.0000


In [81]:
#Set the target(y) and features (X)
X= combined_df_2.drop(columns='weekly_range').values
y= combined_df_2[['weekly_range']].values

In [82]:
X.shape

(1022, 32)

In [83]:
y.shape

(1022, 1)

In [84]:
X.reshape(-1, 1)
y.reshape(-1, 1)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

### Splitting Data Between Training and Testing Sets


In [85]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

### Scaling Data with `MinMaxScaler`

Once the training and test datasets are created, we need to scale the data before training the LSTM model. We will use the `MinMaxScaler` from `sklearn` to scale all values between `0` and `1`.

In [86]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import StandardScaler

# Create a MinMaxScaler object
scaler = StandardScaler()

# Fit the MinMaxScaler object with the training feature data X_train
scaler.fit(X_train)

# Scale the features training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit the MinMaxScaler object with the training target data y_train
scaler.fit(y_train)

# Scale the target training and testing sets
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

### Reshape Features Data for the LSTM Model

In [87]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

## 2. Build and Train the LSTM Model

In this section, we will design a custom LSTM RNN in Keras and fit (train) it using the training data.

We will need to:

1. Define the model architecture in Keras.

2. Compile the model.

3. Fit the model to the training data.

In [88]:
# Define the LSTM RNN model.
lstm_model = Sequential()

dropout_fraction = 0.2
    # Layer 1
lstm_model.add(LSTM(
units=1,
return_sequences=True,
input_shape=(X_train.shape[1], 1)
))
lstm_model.add(Dropout(dropout_fraction))
    # # Layer 2
lstm_model.add(LSTM(units=1))
lstm_model.add(Dropout(dropout_fraction))
    # Output layer
lstm_model.add(Dense(1))


### Compiling the LSTM RNN Model

We will compile the model, using the `adam` optimizer, as loss function, we will use `mean_square_error` since the value we want to predict is continuous.

In [89]:
lstm_model.compile(optimizer="adam", loss="mean_squared_error")

In [11]:
from lstm_4d import window_data, data_splited_scaled, lstm_model, model_fit, lstm_evaluation, lstm_prediction, predicted_plotting 

### Training the Model

Once the model is defined, we train (fit) the model using `100` epochs. Since we are working with time-series data, it's important to set `shuffle=False` since it's necessary to keep the sequential order of the data.

In [90]:
lstm_model.fit(X_train, y_train, epochs=100, shuffle=False, batch_size=1, verbose=1)

Epoch 1/100
715/715 [==============================] - 13s 14ms/step - loss: 1.0211
Epoch 2/100
715/715 [==============================] - 11s 15ms/step - loss: 1.0110
Epoch 3/100
715/715 [==============================] - 9s 13ms/step - loss: 1.0057
Epoch 4/100
715/715 [==============================] - 10s 14ms/step - loss: 1.0036
Epoch 5/100
715/715 [==============================] - 11s 16ms/step - loss: 1.0024
Epoch 6/100
715/715 [==============================] - 12s 17ms/step - loss: 1.0018
Epoch 7/100
715/715 [==============================] - 12s 17ms/step - loss: 1.0012
Epoch 8/100
715/715 [==============================] - 12s 17ms/step - loss: 1.0016
Epoch 9/100
715/715 [==============================] - 12s 17ms/step - loss: 0.9998
Epoch 10/100
715/715 [==============================] - 12s 16ms/step - loss: 1.0003
Epoch 11/100
715/715 [==============================] - 12s 16ms/step - loss: 0.9991
Epoch 12/100
715/715 [==============================] - 12s 17ms/step - los

## 3. Model Performance

In this section, we will evaluate the model using the test data. 

We will need to:

1. Evaluate the model using the `X_test` and `y_test` data.

2. Use the `X_test` data to make predictions.

3. Create a DataFrame of real (`y_test`) vs predicted values.

4. Plot the Real vs predicted values as a line chart.

In [91]:
# Evaluate the model
lstm_model.evaluate(X_test, y_test)

10/10 [==============================] - 1s 4ms/step - loss: 1.8334


1.8334102630615234

In [92]:
prediction = lstm_model(X_test)

In [93]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(prediction)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [97]:
# Create a DataFrame of Real and Predicted values
range = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = combined_df_2.index[-len(real_prices): ])
range.tail()

,Real,Predicted
date,,
2022-03-11,17.1818,4.386003
2022-03-14,0.0000,4.523410
2022-03-15,0.0000,4.595066
2022-03-16,0.0000,4.370622
2022-03-17,0.0000,3.869095


In [96]:
range.hvplot(ylabel='weekly range')

:NdOverlay   [Variable]
   :Curve   [date]   (value)